In [1]:
import pandas as pd
from simple.chart import chartFigure

In [2]:
# resample interval
h = '5min'

In [3]:
# Read tick public trades sample
T = pd.read_parquet('data/20230901.RIU3.tick.parquet')
T

,Price,Size
DateTime,,
2023-09-01 10:00:00.177,106670.0,-1.0
2023-09-01 10:00:00.177,106670.0,-1.0
2023-09-01 10:00:00.177,106670.0,-5.0
2023-09-01 10:00:00.177,106670.0,-1.0
2023-09-01 10:00:00.177,106670.0,-2.0
...,...,...
2023-09-01 18:45:57.933,106530.0,-1.0
2023-09-01 18:45:57.933,106530.0,-1.0
2023-09-01 18:45:57.933,106530.0,-1.0


In [8]:
# Resample public trades to OHLC
C = T.Price.resample(h).ohlc()
C.head()

,open,high,low,close
DateTime,,,,
2023-09-01 10:00:00,106670.0,106680.0,106470.0,106580.0
2023-09-01 10:05:00,106570.0,106620.0,106540.0,106560.0
2023-09-01 10:10:00,106560.0,106600.0,106500.0,106590.0
2023-09-01 10:15:00,106570.0,106650.0,106560.0,106640.0
2023-09-01 10:20:00,106640.0,106650.0,106550.0,106570.0


In [9]:
# Read orderlog sample
L = pd.read_parquet('data/20230901.RIU3.orderlog.parquet')
L

,OrderID,Price,Size,Action,Side
DateTime,,,,,
2023-09-01 10:00:00.050,1925036893624405676,106760.0,6,0,S
2023-09-01 10:00:00.050,1925036893624405749,106750.0,6,1,S
2023-09-01 10:00:00.050,1925036893624405743,106670.0,2,0,B
2023-09-01 10:00:00.080,1925036893624405737,106810.0,4,0,S
2023-09-01 10:00:00.080,1925036893624405750,106690.0,4,1,S
...,...,...,...,...,...
2023-09-01 18:45:57.997,1925036893624992038,106700.0,1,0,S
2023-09-01 18:45:58.103,1925036893625021693,106730.0,1,1,S
2023-09-01 18:45:58.177,1925036893625021694,106550.0,1,1,B


In [10]:
# Split by buy/sell events
Buy, Sell = L[L.Side == 'B'], L[L.Side == 'S']
Buy

,OrderID,Price,Size,Action,Side
DateTime,,,,,
2023-09-01 10:00:00.050,1925036893624405743,106670.0,2,0,B
2023-09-01 10:00:00.080,1925036893624405516,106640.0,1,0,B
2023-09-01 10:00:00.080,1925036893624405752,106540.0,1,1,B
2023-09-01 10:00:00.080,1925036893624405702,106650.0,1,0,B
2023-09-01 10:00:00.080,1925036893624405753,106550.0,1,1,B
...,...,...,...,...,...
2023-09-01 18:45:57.933,1925036893625021679,106430.0,1,1,B
2023-09-01 18:45:57.933,1925036893625021682,106530.0,1,1,B
2023-09-01 18:45:57.933,1925036893625021683,106530.0,1,1,B


In [11]:
# Calculate resampled activities by event counts
Buy_add = Buy[Buy.Action == 1].Action.resample(h).count()
Sell_add = Sell[Sell.Action == 1].Action.resample(h).count()

Buy_cancel = Buy[Buy.Action == 0].Action.resample(h).count()
Sell_cancel = Sell[Sell.Action == 0].Action.resample(h).count()

Buy_cancel.head()

DateTime
2023-09-01 10:00:00    2648
2023-09-01 10:05:00    1832
2023-09-01 10:10:00    2210
2023-09-01 10:15:00    1514
2023-09-01 10:20:00    1597
Freq: 5T, Name: Action, dtype: int64

In [12]:
fig = chartFigure(rows=3, 
        RI=dict(mode='candlestick', x=C.index, open=C.open, high=C.high, low=C.low, close=C.close),
        Add_delta=dict(mode='lines', x=Buy_add.index, y=Buy_add-Sell_add, row=2),
        Cancel_delta=dict(mode='lines', x=Buy_cancel.index, y=Buy_cancel-Sell_cancel, row=3),
                  
)

fig.add_bar(x=Buy_add.index, y=Buy_add.values, name='Buy Add', marker_color='green', opacity=0.35, offsetgroup=0, row=2, col=1)
fig.add_bar(x=Sell_add.index, y=-Sell_add.values, name='Sell Add', marker_color='red', opacity=0.35, offsetgroup=0, row=2, col=1)

fig.add_bar(x=Buy_cancel.index, y=Buy_cancel.values, name='Buy Cancel', marker_color='green', opacity=0.35, offsetgroup=0, row=3, col=1)
fig.add_bar(x=Sell_cancel.index, y=-Sell_cancel.values, name='Sell Cancel', marker_color='red', opacity=0.35, offsetgroup=0, row=3, col=1)

fig.update_xaxes(spikemode='across+marker', spikedash='dot', spikethickness=1, spikesnap='cursor')
fig.update_traces(xaxis='x3')

fig

FigureWidgetResampler({
    'data': [{'close': array([106580., 106560., 106590., 106640., 106570., 106630., 106740., 106720.,
                              106700., 106750., 106720., 106600., 106420., 106490., 106400., 106360.,
                              106340., 106250., 106290., 106260., 106350., 106420., 106460., 106370.,
                              106310., 106350., 106400., 106390., 106390., 106420., 106420., 106500.,
                              106470., 106430., 106320., 106230., 106070., 106040., 106030., 106020.,
                              106130., 106040., 106030., 106020., 106000., 106000., 106170., 106050.,
                                  nan, 105880., 105960., 106020., 106050., 106050., 106120., 106090.,
                              106210., 106250., 106130., 106120., 106140., 106190., 106210., 106180.,
                              106190., 106050., 106170., 106130., 106100., 106120., 106090., 106140.,
                              106180., 106250., 106360., 1